## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [309]:
# Import libraries.
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime


In [310]:
# Load up store_income_data.csv
store_df = pd.read_csv('store_income_data_task.csv')

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [311]:
# Check for all unique countries. Bonus: Check amount of unique countries.
print("Number of unique countries: ", store_df['country'].nunique())
store_df['country'].unique()

Number of unique countries:  76


array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

In [312]:
# Convert the country column to lowercase. 
store_df['country'] = store_df['country'].str.lower()

# Delete white spaces.
store_df['country'] = store_df['country'].str.strip()

# Check for changes.
print("Number of unique countries: ", store_df['country'].nunique())
store_df['country'].unique()

Number of unique countries:  36


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [313]:
def replace_matches_in_column(df, column, string_to_match, min_ratio=70):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = process.extract(string_to_match, strings, limit=10, scorer=fuzz.token_sort_ratio)

    # Only get matches with a ratio >= min_ratio
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input string_to_match
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    print("All done!")

In [314]:
# Choose the variations you want.

replace_matches_in_column(df=store_df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=store_df, column='country', string_to_match="uk")
replace_matches_in_column(df=store_df, column='country', string_to_match="england")
replace_matches_in_column(df=store_df, column='country', string_to_match="britain")
replace_matches_in_column(df=store_df, column='country', string_to_match="united states")
replace_matches_in_column(df=store_df, column='country', string_to_match="united states of america")
replace_matches_in_column(df=store_df, column='country', string_to_match="america")
replace_matches_in_column(df=store_df, column='country', string_to_match="south africa")
replace_matches_in_column(df=store_df, column='country', string_to_match="sa")


All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!


In [315]:
# Check progress
print("Number of unique countries: ", store_df['country'].nunique())
store_df['country'].unique()

Number of unique countries:  17


array(['united states of america', 'britain', 'united kingdom', 'u.k.',
       'sa', 'u.k/', 'america', nan, 's.a.', 'england', 'uk', 's.a./',
       'u.k', '', 's.a..', 'south africa', '/', '.'], dtype=object)

In [316]:
# Clean up the rest of the differences.
# Note: Despite decreasing the min_ratio, the program would not find matches for 'uk' etc.

store_df.replace('uk', 'united kingdom', inplace=True)
store_df.replace('u.k.', 'united kingdom', inplace=True)
store_df.replace('u.k/', 'united kingdom', inplace=True)
store_df.replace('u.k', 'united kingdom', inplace=True)
store_df.replace('britain', 'united kingdom', inplace=True)
store_df.replace('england', 'united kingdom', inplace=True)

store_df.replace('united states of america', 'united states', inplace=True)
store_df.replace('america', 'united states', inplace=True)

store_df.replace('s.a.', 'south africa', inplace=True)
store_df.replace('s.a./', 'south africa', inplace=True)
store_df.replace('s.a..', 'south africa', inplace=True)
store_df.replace('sa', 'south africa', inplace=True)

# Replace non-values with np.nan.
store_df.replace('/', np.nan, inplace=True)
store_df.replace('.', np.nan, inplace=True)
store_df.replace('', np.nan, inplace=True)

# Show progress.
print("Number of unique countries: ", store_df['country'].nunique())
store_df['country'].unique()

Number of unique countries:  3


array(['united states', 'united kingdom', 'south africa', nan],
      dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [317]:
# Determine datatype.
print(store_df['date_measured'].dtype)

# Check date first rows to understand date time format.
store_df.head()

object


,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united kingdom
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united kingdom
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united kingdom


In [318]:
# Create copy of days_ago column.
# Turn that copy into the correct date_time format.
# Subtract it from today's date.
# Note: Using 'datetime.date.today()' did not work for me.
store_df['days_ago'] = (datetime.now() - pd.to_datetime(store_df['date_measured'], format= '%d-%m-%Y')).dt.days
# Check for new column.
store_df.head()

,id,store_name,store_email,department,income,date_measured,country,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united states,6964
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united kingdom,6995
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united states,7840
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united kingdom,6871
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united kingdom,19031
